导入了一些常用的数据处理库和函数，为后续的数据加载和数据增强做准备。

In [1]:
import numpy as np  # 用于科学计算
from batchgenerators.dataloading.data_loader import DataLoader  # 用于加载训练数据和标签数据
from batchgenerators.transforms.abstract_transforms import Compose  # 用于将多个数据变换组合在一起
from batchgenerators.transforms.spatial_transforms import (
    MirrorTransform,  # 镜像变换（左右镜像、上下镜像）
    SpatialTransform,  # 空间变换（旋转、缩放、平移）
)
from batchgenerators.transforms.color_transforms import (
    BrightnessMultiplicativeTransform,  # 亮度变换（乘性）
    ContrastAugmentationTransform,  # 对比度变换（增强）（线性）（非线性）
)
from batchgenerators.transforms.noise_transforms import (
    GaussianNoiseTransform,  # 高斯噪声变换
    GaussianBlurTransform,  # 高斯模糊变换
)
from batchgenerators.augmentations.crop_and_pad_augmentations import crop  # 裁剪和填充变换

定义函数 `get_split_fold` 接收一个 csv 文件的数据，该数据集已按照交叉验证的折叠进行了拆分，其中训练集为第 0 折，测试集 A 为第 1 折，测试集 B 为第 2 折。函数通过查找 "fold" 列的值来确定每个数据点所属的数据集，然后返回一个包含训练、测试 A 和测试 B 数据集的字典。

In [2]:
def get_split_fold(data):
    """
    如果数据集已经按照 [0,1,2] 分成了三个数据集
    其中:
    - 训练集 => 0
    - 测试集A => 1
    - 测试集B => 2
    @param data: 存储数据集的 CSV 文件
    @return: 训练集，测试集A，测试集B 的字典
    """
    # 折叠数据的返回索引
    train_idx = np.where(data["fold"] == 0)[0]
    testA_idx = np.where(data["fold"] == 1)[0]
    testB_idx = np.where(data["fold"] == 2)[0]

    # 为每个数据集创建字典
    train_ds = {
        "img_npy": [data["img_npy"].tolist()[i] for i in train_idx],
        "anno_npy": [data["anno_npy"].tolist()[i] for i in train_idx],
        "patient_id": [data["patient ID"].tolist()[i] for i in train_idx],
    }
    testA_ds = {
        "img_npy": [data["img_npy"].tolist()[i] for i in testA_idx],
        "anno_npy": [data["anno_npy"].tolist()[i] for i in testA_idx],
        "patient_id": [data["patient ID"].tolist()[i] for i in testA_idx],
    }
    testB_ds = {
        "img_npy": [data["img_npy"].tolist()[i] for i in testB_idx],
        "anno_npy": [data["anno_npy"].tolist()[i] for i in testB_idx],
        "patient_id": [data["patient ID"].tolist()[i] for i in testB_idx],
    }

    return {"train_ds": train_ds, "testA_ds": testA_ds, "testB_ds": testB_ds}

定义函数 `get_train_transform(patch_size, prob)` ，参数1为输入图像的大小，输入2为变换的概率。该函数返回一个数据增强的变换列表，这些变换用于增强神经网络模型训练数据。这里采用的数据增强方法包括：弹性变形、镜像变换、亮度调整、高斯噪声、高斯模糊和对比度增强。其中弹性变形可以减少数据集的大小，并且不会引入边界伪影。

In [3]:
def get_train_transform(patch_size, prob=0.5):
    tr_transforms = []  # 创建一个空列表，用于存储变换
    # 使用SpatialTransform进行空间变换
    tr_transforms.append(
        SpatialTransform(
            patch_size,  # 输入图像的大小
            [i // 2 for i in patch_size],  # 中心点
            do_elastic_deform=True,  # 弹性变形
            alpha=(0.0, 300.0),  # 弹性变形的强度
            sigma=(20.0, 40.0),  # 弹性变形的平滑度
            do_rotation=True,  # 旋转
            angle_x=(-np.pi / 15.0, np.pi / 15.0),  # 旋转角度
            angle_y=(-np.pi / 15.0, np.pi / 15.0),  # 旋转角度
            angle_z=(0.0, 0.0),  # 旋转角度
            do_scale=True,  # 缩放
            scale=(1 / 1.15, 1.15),  # 缩放比例
            random_crop=False,  # 随机裁剪
            border_mode_data="constant",  # 边界模式：常数
            border_cval_data=0,  # 边界值
            order_data=3,  # 数据的阶数
            p_el_per_sample=prob,  # 弹性变形的概率
            p_rot_per_sample=prob,  # 旋转的概率
            p_scale_per_sample=prob,  # 缩放的概率
        )
    )
    # 使用MirrorTransform进行镜像变换，这里只进行左右镜像
    tr_transforms.append(MirrorTransform(axes=(1,)))
    # 使用BrightnessMultiplicativeTransform对图像的亮度进行调整
    tr_transforms.append(
        # 下面的三个参数分别是：亮度的乘性因子，是否对每个通道进行变换，每个样本的变换概率
        BrightnessMultiplicativeTransform(
            (0.7, 1.5), per_channel=True, p_per_sample=prob
        )
    )
    # 使用GaussianNoiseTransform对图像添加高斯噪声，噪声的方差在 [0, 0.5] 之间
    tr_transforms.append(
        GaussianNoiseTransform(noise_variance=(0, 0.5), p_per_sample=prob)
    )
    # 使用GaussianBlurTransform对图像进行高斯模糊，模糊的程度在 [0.5, 2.0] 之间
    tr_transforms.append(
        GaussianBlurTransform(
            blur_sigma=(0.5, 2.0),
            different_sigma_per_channel=True,
            p_per_channel=prob,
            p_per_sample=prob,
        )
    )
    # 使用ContrastAugmentationTransform对图像进行对比度增强，增强的程度在 [0.75, 1.25] 之间
    tr_transforms.append(
        ContrastAugmentationTransform(contrast_range=(0.75, 1.25), p_per_sample=prob)
    )
    # 使用Compose将这些变换组合在一起
    tr_transforms = Compose(tr_transforms)
    return tr_transforms  # 返回变换

继承 `batchgenerators.dataloading.data_loader.DataLoader` 定义自己的 `DataLoader` 类，用于加载训练数据，并生成训练批次。构造函数接收训练数据、批量大小、裁剪大小、线程数量等参数，并可选择是否启用数据洗牌、数据增强等功能。

In [4]:
class DataLoader(DataLoader):  # batchgenerators.dataloading.data_loader.DataLoader
    def __init__(
        self,
        data,  # 数据集：必须是由get_list_of_patients返回的患者列表（并由get_split_deterministic拆分）
        batch_size,  # 批次大小
        patch_size,  # 批具有的空间大小
        num_threads_in_multithreaded,  # 多线程
        crop_status=False,  # 是否裁剪：默认不裁剪
        crop_type="center",  # 裁剪类型：中心裁剪
        seed_for_shuffle=1234,  # 随机种子
        return_incomplete=False,  # 默认不返回不完整的批次
        shuffle=True,  # 默认打乱
        infinite=True,  # 默认无限循环
        margins=(0, 0, 0),  # 边距0
    ):
        super().__init__(
            data,
            batch_size,
            num_threads_in_multithreaded,
            seed_for_shuffle,
            return_incomplete,
            shuffle,
            infinite,
        )
        self.patch_size = patch_size  # 批具有的空间大小
        self.n_channel = 3  # 通道数
        self.indices = list(range(len(data["img_npy"])))  # 索引
        self.crop_status = crop_status  # 是否裁剪
        self.crop_type = crop_type  # 裁剪类型
        self.margins = margins  # 边距

    @staticmethod  # 静态方法：加载患者
    def load_patient(img_path):
        img = np.load(img_path, mmap_mode="r")  # 以只读模式加载图像
        return img

    def generate_train_batch(self):
        idx = self.get_indices()  # 调用父类的方法获取下一个批次中要使用的病人的索引
        gland_img = [self._data["img_npy"][i] for i in idx]  # 根据索引获取数据图像
        img_seg = [self._data["anno_npy"][i] for i in idx]  # 根据索引获取标注图像
        patient_id = [self._data["patient_id"][i] for i in idx]  # 根据索引获取病人ID
        # 初始化空数组用于存储数据和标注
        img = np.zeros(
            (len(gland_img), self.n_channel, *self.patch_size), dtype=np.float32
        )
        seg = np.zeros(
            (len(img_seg), self.n_channel, *self.patch_size), dtype=np.float32
        )
        # 迭代patients_for_batch并将其包含在批次中
        for i, (j, k) in enumerate(zip(gland_img, img_seg)):
            img_data = self.load_patient(j)  # 加载数据图像
            seg_data = self.load_patient(k)  # 加载标注图像
            # 根据文档要求，输入图像应该以通道为首的顺序输入，因此我们使用张量操作来转换为通道为首的格式
            img_data = np.einsum("hwc->chw", img_data)
            seg_data = np.einsum("hwc->chw", seg_data)
            # 现在随机裁剪到self.patch_size大小
            # crop期望数据为(b, c, x, y, z)，但patient_data的形状为(c, x, y, z)，因此我们需要添加一个虚拟维度，以便它能够工作（@Todo，可以改进）
            if self.crop_status:
                img_data, seg_data = crop(
                    img_data[None],
                    seg=seg_data[None],
                    crop_size=self.patch_size,
                    margins=self.margins,
                    crop_type=self.crop_type,
                )
                img[i] = img_data[0]
                seg[i] = seg_data[0]
            else:
                img[i] = img_data
                seg[i] = seg_data
        return {"data": img, "seg": seg, "patient_id": patient_id}

定义一系列工具函数，用于图像处理和分割。

In [5]:
# 导入matplotlib和skimage模块以支持图像处理和分割。
import matplotlib.pyplot as plt
from skimage import color
from skimage import segmentation

定义函数 plot_comparison 用于绘制多个图像的比较，但仅在列方向上进行。输入参数包括图像列表、标题列表、行数、列数、绘图标识、保存路径等。如果绘图标识为True，函数将绘制图像；否则，它将返回一个图像对象。

In [6]:
def plot_comparison(
    input_img,  # 输入图像
    caption=None,  # 标题
    plot=True,  # 是否绘制
    save_path=None,  # 保存路径
    save_name=None,  # 保存名称
    save_as="png",  # 保存格式
    save_dpi=300,  # 保存分辨率
    captions_font=20,  # 标题字体大小
    n_row=1,  # 行数
    n_col=2,  # 列数
    figsize=(5, 5),  # 图像大小
    cmap="gray",  # 颜色映射：灰度
):
    print()
    if caption is not None:
        assert len(caption) == len(
            input_img
        ), "Caption length and input image length does not match"
    assert len(input_img) == n_col, "Error of input images or number of columns!"

    fig, axes = plt.subplots(n_row, n_col, figsize=figsize)
    fig.subplots_adjust(hspace=0.4, wspace=0.4, right=0.7)

    for i in range(n_col):
        axes[i].imshow(np.squeeze(input_img[i]), cmap=cmap)
        if caption is not None:
            axes[i].set_xlabel(caption[i], fontsize=captions_font)
        axes[i].set_xticks([])
        axes[i].set_yticks([])

    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path + "{}.{}".format(save_name, save_as), save_dpi=save_dpi)
    if plot:
        plt.show()
    else:
        return fig

定义函数 plot_hist 用于绘制两个图像的直方图，这两个图像在同一行中，并且标题显示在图像下方。输入参数包括图像列表、标题列表、行数、列数、bin数、像素值范围等。

In [7]:
def plot_hist(
    inp_img, titles, n_row=1, n_col=2, n_bin=20, ranges=[0, 1], figsize=(5, 5)
):
    """输入图像的绘制直方图
    Args:
        inp_img (_type_): 图像列表
        titles (_type_): 标题列表
        n_row (int, optional): 行数
        n_col (int, optional): 列数
        n_bin (int, optional): bin数
        ranges (list, optional): 范围
        figsize (tuple, optional): 图像大小
    """
    assert len(titles) == len(
        inp_img
    ), "Caption length and input image length does not match"
    assert len(inp_img) == n_col, "Error of input images or number of columns!"

    fig, axes = plt.subplots(n_row, n_col, figsize=figsize)
    fig.subplots_adjust(hspace=0.4, wspace=0.4, right=0.7)

    for i in range(n_col):
        inp = np.squeeze(inp_img[i])
        axes[i].hist(inp.ravel(), n_bin, ranges)
        axes[i].set_title(titles[i])
        axes[i].set_xlabel("Pixel Value")
        axes[i].set_ylabel("Frequency")

    plt.tight_layout()
    plt.show()

定义函数 overlay_mask 将分割掩模覆盖在原始图像上。输入参数包括图像、分割掩模、颜色和掩模不透明度。

In [8]:
def overlay_mask(image, mask, colors=[(0, 1.0, 0)], alpha=0.12):
    # 图像归一化
    if np.max(image) > 1.0:
        image = image / 255.0
    # 灰度图像
    if mask.ndim == 3:
        mask = mask[:, :, 0]
    mask_image = color.label2rgb(mask, image, colors=colors, alpha=alpha, bg_label=0)
    return mask_image

overlay_boundary 函数在原始图像上绘制分割边界线。输入参数包括图像、分割掩模、颜色和边界线模式。

In [9]:
def overlay_boundary(image, mask, color=(0, 1.0, 0), mode="thick"):
    if np.max(image) > 1.0:
        image = image / 255.0
    if mask.ndim == 3:
        mask = mask[:, :, 0]
    boundary_image = segmentation.mark_boundaries(image, mask, color=color, mode=mode)
    return boundary_image

plot_labels_color 通过循环使用matplotlib定义的颜色映射可视化分割标签。输入参数包括标签、颜色映射等。

In [10]:
def plot_labels_color(label_im, cmap="tab20c"):
    # 构造彩色图像以叠加
    color_mask = np.zeros(label_im.shape)
    get_cmap = plt.cm.get_cmap(cmap)
    # 循环通过cmap为每种颜色是相关联的标签
    for i in range(np.max(label_im)):
        color_mask[label_im[:, :, 0] == i + 1] = list(get_cmap(i))[:-1]
    return color_mask

In [11]:
# 用于输入图像的通道方向最小最大归一化
def min_max_norm(img, axis=(1, 2)):
    inp_shape = img.shape
    img_min = np.broadcast_to(img.min(axis=axis, keepdims=True), inp_shape)
    img_max = np.broadcast_to(img.max(axis=axis, keepdims=True), inp_shape)
    x = (img - img_min) / (img_max - img_min + float(1e-18))
    return x

In [12]:
import warnings
import os
import pandas as pd
from batchgenerators.dataloading.multi_threaded_augmenter import (
    MultiThreadedAugmenter,
)  # 多线程数据增强数据加载器
import wandb
from tqdm import tqdm  # 显示进度条
import torch  # PyTorch深度学习库
from torch import nn  # 神经网络模块
import segmentation_models_pytorch as smp  # 用于图像分割的PyTorch中的模型
from torchsummary import summary  # 打印模型概要信息
import math
import time

c:\Software\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. 忽略Python警告，以确保代码更加简洁、可读和易于维护。
2. 使用WandB库初始化一个新的项目并将其与Gland_Seg项目中的glaseg实体关联起来。
3. 从指定的CSV文件中读取数据，并将其转化为数据集字典形式。
4. 根据配置文件中指定的参数，获取patch_size、batch_size和epochs。
5. 获取训练数据集的数据增强变换和使用MultiThreadedAugmenter类对数据进行扩增。
6. 使用DataLoader类对训练集和验证集进行分批处理，设置batch_size、patch_size、num_threads_in_multithreaded等参数，并使用了pin_memory=False命令禁用固定内存功能。

In [13]:
warnings.filterwarnings("ignore")
# wandb.init(project="Gland_Seg", entity="glaseg", config="config/config.yaml")
wandb.init(project="Gland_Seg", config="config/config.yaml")
config = wandb.config
tabular_data = pd.read_csv(config.csv)
ds_dict = get_split_fold(tabular_data)
patch_size = eval(config.patch_size)
batch_size = config.batch_size
epochs = config.epochs
tr_transforms = get_train_transform(patch_size, prob=config.aug_prob)
train_dl = DataLoader(
    data=ds_dict["train_ds"],
    batch_size=batch_size,
    patch_size=patch_size,
    num_threads_in_multithreaded=4,
    seed_for_shuffle=5243,
    return_incomplete=False,
    shuffle=True,
    infinite=True,
)
train_gen = MultiThreadedAugmenter(
    train_dl,
    tr_transforms,
    num_processes=4,
    num_cached_per_queue=2,
    seeds=None,
    pin_memory=False,
)
val_dl = DataLoader(
    data=ds_dict["testA_ds"],
    batch_size=batch_size,
    patch_size=patch_size,
    num_threads_in_multithreaded=1,
    seed_for_shuffle=5243,
    return_incomplete=False,
    shuffle=True,
    infinite=True,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xftxyz2001. Use `wandb login --relogin` to force relogin


1. 使用 `smp.Unet` 模型定义一个深度学习模型，并配置所需参数，然后定义一个优化器并指定学习率。
2. 定义一个学习率调度器，用于在训练过程中动态调整学习率。（调度器在验证集上监测到模型性能没有提高时，就将学习率减少一个倍数。）
3. 使用标签平滑技术定义一个损失函数。

In [14]:
# 获取用于训练的 CPU 或 GPU 设备
device = "cuda" if torch.cuda.is_available() else "cpu"

# 定义模型
model = smp.Unet(
    encoder_name=config.encoder_model,
    decoder_use_batchnorm=True,
    in_channels=3,
    classes=config.n_class,
).to(device)

# 打印模型概述
summary(model, (3, 512, 512))
# 定义优化器
optimizer = eval(config.optimizer)(model.parameters(), lr=float(config.learning_rate))

# 定义学习率调度器
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=20, factor=0.1
)

# 定义 Dice 损失函数
dice_loss = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)

# 定义带有标签平滑的交叉熵损失函数
xent = smp.losses.SoftBCEWithLogitsLoss(smooth_factor=0.1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           9,408
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5         [-1, 64, 128, 128]          36,864
       BatchNorm2d-6         [-1, 64, 128, 128]             128
              ReLU-7         [-1, 64, 128, 128]               0
            Conv2d-8         [-1, 64, 128, 128]          36,864
       BatchNorm2d-9         [-1, 64, 128, 128]             128
             ReLU-10         [-1, 64, 128, 128]               0
       BasicBlock-11         [-1, 64, 128, 128]               0
           Conv2d-12         [-1, 64, 128, 128]          36,864
      BatchNorm2d-13         [-1, 64, 128, 128]             128
             ReLU-14         [-1, 64, 1

自定义一个损失函数 custom_loss 接受模型的预测值和目标值作为输入，通过计算二分类交叉熵和Dice loss来计算总损失，并返回总损失、二分类交叉熵和Dice loss三个值。

In [15]:
def custom_loss(pred, target):
    xent_l = xent(pred, target)
    dice_l = dice_loss(pred, target)
    loss = xent_l + dice_l
    return loss, xent_l, dice_l

接受模型和优化器作为输入，通过训练集数据计算总损失、二分类交叉熵和Dice loss，并返回这些值，以及输入图像、目标掩模和模型的预测掩模。

In [16]:
def train(model, optimizer):
    # total number of training batches
    num_batches = math.ceil(len(ds_dict["train_ds"]["img_npy"]) / batch_size)
    model.train()
    batch_xent_l = []
    batch_dice_l = []
    batch_loss = []
    print("Training...")
    for i in tqdm(range(num_batches)):
        train_batch = next(train_gen)
        imgs = train_batch["data"]
        segs = train_batch["seg"]
        # normalization
        imgs = min_max_norm(imgs)
        # binarisation
        segs = np.where(segs > 0.0, 1.0, 0.0).astype("float32")
        segs = np.expand_dims(segs[:, 0, :, :], 1)
        imgs, segs = torch.from_numpy(imgs).to(device), torch.from_numpy(segs).to(
            device
        )
        # Compute loss
        pred = model(imgs)
        loss, xent_l, dice_l = custom_loss(pred, segs)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # batch losses
        batch_xent_l.append(xent_l)
        batch_dice_l.append(dice_l)
        batch_loss.append(loss)
    # apply sigmoid to masking
    segs = nn.Sigmoid()(segs)
    # taking the average along the batch
    loss = torch.mean(torch.as_tensor(batch_loss)).item()
    avg_xent_l = torch.mean(torch.as_tensor(batch_xent_l)).item()
    avg_dice_l = torch.mean(torch.as_tensor(batch_dice_l)).item()

    return {
        "loss": loss,
        "xent_l": avg_xent_l,
        "dice_l": avg_dice_l,
        "imgs": imgs.cpu().detach().numpy(),
        "segs": segs.cpu().detach().numpy(),
        "pred": pred.cpu().detach().numpy(),
    }


接受模型作为输入，通过测试集数据计算总损失、二分类交叉熵和Dice loss，并返回这些值，以及输入图像、目标掩模和模型的预测掩模。

In [17]:
def test(model):
    num_batches = math.ceil(len(ds_dict["testA_ds"]["img_npy"]) / batch_size)
    model.eval()
    # no need back prop for testing set
    batch_xent_l = []
    batch_dice_l = []
    batch_loss = []
    print("Testing...")
    with torch.no_grad():
        for i in tqdm(range(num_batches)):
            val_batch = next(val_dl)
            imgs = val_batch["data"]
            segs = val_batch["seg"]
            # normalization
            imgs = min_max_norm(imgs)
            # binarisation
            segs = np.where(segs > 0.0, 1.0, 0.0).astype("float32")
            segs = np.expand_dims(segs[:, 0, :, :], 1)
            imgs, segs = torch.from_numpy(imgs).to(device), torch.from_numpy(segs).to(
                device
            )
            # Compute loss
            pred = model(imgs)
            loss, xent_l, dice_l = custom_loss(pred, segs)
            # batch losses
            batch_xent_l.append(xent_l)
            batch_dice_l.append(dice_l)
            batch_loss.append(loss)
        # apply sigmoid to masking
        segs = nn.Sigmoid()(segs)
        # taking the average along the batch
        loss = torch.mean(torch.as_tensor(batch_loss)).item()
        avg_xent_l = torch.mean(torch.as_tensor(batch_xent_l)).item()
        avg_dice_l = torch.mean(torch.as_tensor(batch_dice_l)).item()
    return {
        "loss": loss,
        "xent_l": avg_xent_l,
        "dice_l": avg_dice_l,
        "imgs": imgs.cpu().detach().numpy(),
        "segs": segs.cpu().detach().numpy(),
        "pred": pred.cpu().detach().numpy(),
    }


在训练和测试函数中，首先将输入数据进行归一化和二值化处理，然后将处理后的数据转换为PyTorch张量，将其输入到模型中进行预测，并计算损失。然后使用反向传播算法计算梯度，更新模型参数。最后，将损失和其他指标取平均值，并将所有张量转换为numpy数组，以便进行可视化和进一步分析。

主函数逻辑：
1. 定义两个变量current_total_loss和current_dice_score，分别用于记录当前最佳的总损失值和dice得分。
2. 对于每个epoch进行循环，从1到epochs+2。
3. 输出当前epoch的编号。
4. 调用train()函数进行模型训练，返回train_output，包括总损失值、二元交叉熵损失值和dice得分。
5. 调用test()函数对模型进行测试，返回test_output，包括总损失值、二元交叉熵损失值、dice得分、模型预测输出和原始图像和标签。
6. 调用scheduler.step()函数来更新学习率，scheduler是一个torch.optim.lr_scheduler.ReduceLROnPlateau类型的对象，用于动态调整学习率。
7. 打印训练输出结果和验证输出结果，分别包括总损失值、二元交叉熵损失值和dice得分。
8. 使用wandb.log()函数记录日志，包括训练集和验证集的总损失值、二元交叉熵损失值和dice得分，以及当前的学习率。
9. 如果当前epoch是10的倍数，则进行预测可视化并使用wandb.log()函数记录可视化结果。
10. 如果当前的验证集总损失值小于current_total_loss，则保存当前模型的参数，并将current_total_loss更新为当前验证集总损失值。同样的，如果当前dice得分大于current_dice_score，则保存当前模型的参数，并将current_dice_score更新为当前dice得分。
11. 循环结束后，打印模型的训练时间。

In [18]:
def main():
    start = time.time()
    current_total_loss = 1000
    current_dice_score = 0
    for e in range(1, epochs + 2):
        print("Epcohs:", e)
        train_output = train(model, optimizer)
        test_output = test(model)
        scheduler.step(test_output["loss"])
        print("Training Outputs: ")
        print(
            "Total loss: {:.2f}, BCE: {:.2f}, Dice Score: {:.2f}".format(
                train_output["loss"], train_output["xent_l"], 1 - train_output["dice_l"]
            )
        )
        print("-" * 100)
        print("Validation Outputs: ")
        print(
            "Total loss: {:.2f}, BCE: {:.2f}, Dice Score: {:.2f}".format(
                test_output["loss"], test_output["xent_l"], 1 - test_output["dice_l"]
            )
        )
        # logging
        wandb.log(
            {
                "Train_total_loss": train_output["loss"],
                "Val_total_loss": test_output["loss"],
            },
            step=e,
        )
        wandb.log(
            {
                "Train_BCE_loss": train_output["xent_l"],
                "Val_BCE_loss": test_output["xent_l"],
            },
            step=e,
        )
        wandb.log(
            {
                "Train_dice_score": 1 - train_output["dice_l"],
                "Val_dice_score": 1 - test_output["dice_l"],
            },
            step=e,
        )
        wandb.log({"Learning rate": optimizer.param_groups[0]["lr"]}, step=e)
        if e % 10 == 0:
            # threshold sigmoid output with 0.5
            pred_thr = np.where(test_output["pred"] > 0.5, 1.0, 0.0)
            # sample a dataset from the batch for visualization purpose
            imgs = [
                test_output["imgs"][0, 0, :, :],
                test_output["segs"][0, 0, :, :],
                pred_thr[0, 0, :, :],
            ]
            captions = ["Gland Image", "Masking", "Prediction"]
            fig = plot_comparison(
                imgs,
                captions,
                plot=False,
                n_col=len(imgs),
                figsize=(12, 12),
                cmap="gray",
            )
            wandb.log({"Validation Dataset Output Sample": wandb.Image(fig)}, step=e)

        # save model
        weights_dir = "./weights/"
        if not os.path.exists(weights_dir):
            os.makedirs(weights_dir)
        base_path = os.path.split(weights_dir)[0]
        if test_output["loss"] < current_total_loss:
            current_total_loss = test_output["loss"]
            torch.save(model.state_dict(), weights_dir + "best_loss_{}.pth".format(e))
            wandb.save(
                os.path.join(weights_dir, "best_loss_{}.pth".format(e)),
                base_path=base_path,
            )
        if (1 - test_output["dice_l"]) > current_dice_score:
            current_dice_score = 1 - test_output["dice_l"]
            torch.save(model.state_dict(), weights_dir + "best_dice_{}.pth".format(e))
            wandb.save(
                os.path.join(weights_dir, "best_dice_{}.pth".format(e)),
                base_path=base_path,
            )
        print()

    print("Model training runtime: {} mins".format((time.time() - start) / 60.0))

In [19]:
if __name__ == "__main__":
    main()


Epcohs: 1
Training...


  0%|          | 0/43 [00:00<?, ?it/s]Exception in thread Thread-6 (results_loop):
Traceback (most recent call last):
  File "c:\Software\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Software\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Software\Python311\Lib\site-packages\batchgenerators\dataloading\multi_threaded_augmenter.py", line 92, in results_loop
    raise RuntimeError("One or more background workers are no longer alive. Exiting. Please check the print"
RuntimeError: One or more background workers are no longer alive. Exiting. Please check the print statements above for the actual error message
  0%|          | 0/43 [00:06<?, ?it/s]


RuntimeError: One or more background workers are no longer alive. Exiting. Please check the print statements above for the actual error message